Name:

Student ID: 

# Lecture Worksheet A-2: Data Wrangling with dplyr (20 pts)

By the end of this worksheet, you will be able to: 

1. Use the five core dplyr verbs for data wrangling: `select()`, `filter()`, `arrange()`, `mutate()`, `summarise()`.
2. Use piping when implementing function chains.
3. Use `group_by()` to operate within groups (of rows) with `mutate()` and `summarise()`. 
4. Use `across()` to operate on multiple columns with `summarise()` and `mutate()`.

## Instructions + Grading

+ Worksheets are autograded like a regular assignment. You will be able to immediately see if you got the question correct, and have unlimited attempts on every questions. **Each question is worth one point, and there are 20 questions.**

+ Autograded questions are easily identifiable through their labelling as **QUESTION**. Any other instructions that prompt the student to write code are activities, which are not graded and thus do not contribute to marks - but do contribute to the workflow of the worksheet!

+ Please save your work regularly and run the worksheet from top to bottom (run all) to ensure your responses are correct.

+ Worksheets are be submitted (as a .ipynb file) on Canvas. See the Syllabus for the late policy. 


## Five core dplyr verbs: an overview of this worksheet

So far, we've **looked** at our dataset. It's time to **work with** it! Prior to creating any models, or using visualization to gain more insights about our data, it is common to tweak the data in some ways to make it a little easier to work with. For example, you may need to rename some variables, reorder observations, or even create some new variables from your existing ones!

As explained in depth in the [R4DS Data Transformation chapter](https://r4ds.had.co.nz/transform.html), there are five key dplyr functions that allow you to solve the vast majority of data manipulation tasks:

+ Pick variables by their names (`select()`)
+ Pick observations by their values (`filter()`)
+ Reorder the rows (`arrange()`)
+ Create new variables with functions of existing variables (`mutate()`)
+ Collapse many rows down to a single summary (`summarise()`)

We can use these in conjunction with two other functions:

- The `group_by()` function groups a tibble by rows. Downstream calls to `mutate()` and `summarise()` operate independently on each group.
- The `across()` function, when used within the `mutate()` and `summarise()` functions, operate on multiple columns.

Because data wrangling involves calling multiple of these functions, we will also see the pipe operator `%>%` for putting these together in a single statement.  

## Getting Started

Load the required packages for this worksheet:

In [ ]:
suppressPackageStartupMessages(library(palmerpenguins))
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(gapminder))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(testthat))
suppressPackageStartupMessages(library(digest))
expect_sorted <- function(object) {
  act <- quasi_label(rlang::enquo(object), arg = "object")
  expect(
    !is.unsorted(act$val),
    sprintf("%s not sorted", act$lab)
  )
  invisible(act$val)
}

The following code chunk has been unlocked, to give you the flexibility to start this document with some of your own code if needed.

In [ ]:
# An unlocked code chunk.

# Part 1: The Five Verbs

## Exploring your data

What's the first thing that you should do when you're starting a project with a new dataset? Having a coffee is a reasonable answer, but before that, you should **look at the data**. This may sound obvious, but a common mistake is to dive into the analysis too early before being familiar with the data - only to have to go back to the start when something goes wrong and you can't quite figure out why. Some of the questions you may want to ask are:

+ What is the format of the data?
+ What are the dimensions?
+ Are there missing data?

You will learn how to answer these questions and more using dplyr.

## Penguins Data

[Palmer penguins](https://github.com/allisonhorst/palmerpenguins) is an R data package created by Allison Horst. Data were collected and made available by Dr. Kristen Gorman and the Palmer Station, Antarctica LTER, a member of the Long Term Ecological Research Network. The dataset that we will be using is stored in a variable called "penguins". It is a subset of the "penguins_raw" dataset, also included in this R package. Let's have a look at it.

In [ ]:
head(penguins)

`head()` returns the first 6 rows of a dataframe, instead of printing all the data to screen.

## What is the format of the data?

Let's begin by checking the class of the **penguins** variable. This will give us a clue about the overall structure of the data.

In [ ]:
class(penguins)

As you can see, the function returns 3 classes: "tbl_df", "tbl", and "data.frame". A dataframe is the default class for data read into R. Tibbles ("tbl" and "tbl_df") are a modern take on data frames, but slightly tweaked to work better in the tidyverse. For now, you don’t need to worry about the differences; we’ll come back to tibbles later. The dataset that we are working with was originally a data.frame that has been coerced into a tibble, which is why multiple class names are returned by the `class()` function.

## What are the dimensions?

There are two functions that we can use to see exactly how many rows (observations) and columns (variables) we're dealing with. `dim()` is the base R option, and `glimpse()` is the dplyr flavour, which gives us some more information besides the row and column number. Give both a try!

In [ ]:
dim(penguins)
glimpse(penguins)

There are more functions that you can use to further explore the dimensions, such as `nrow()`, `ncol()`, `colnames()` or `rownames()`, but we won't be looking into those.

## QUESTION 1.0
{points: 1}

In the `dim()` function, what is the first number that you see?

Multiple choice!

A) number of rows   

B) number of columns

Put your selection (e.g. the letter corresponding to the correct option) into a variable named `answer1.0`. 

*Hint: make sure your answer is a string, such as* `answer1.0 <- "X"`

In [ ]:
# answer1.0 <- "FILL_THIS_IN"
# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
library(digest)
stopifnot("type of answer1.0 is not character"= setequal(digest(paste(toString(class(answer1.0)), "d68a1")), "f51b846d841da7ce219b3076f673c0f6"))
stopifnot("length of answer1.0 is not correct"= setequal(digest(paste(toString(length(answer1.0)), "d68a1")), "272543d77a2d878f30cda11824fbc20b"))
stopifnot("value of answer1.0 is not correct"= setequal(digest(paste(toString(tolower(answer1.0)), "d68a1")), "b6e90345b8a4adabda941aa1169ced83"))
stopifnot("letters in string value of answer1.0 are correct but case is not correct"= setequal(digest(paste(toString(answer1.0), "d68a1")), "85d87a064f382e336e3b06b48c7a2da9"))

print('Success!')

## `select()` 

*A brief interlude on naming things:* Names are important. Jenny Bryan has some excellent [slides](https://speakerdeck.com/jennybc/how-to-name-files) for naming things in a way that is human readable *and* machine readable. Don't worry too much about it for this worksheet, but do keep it in mind as it helps with *reproducibility*. 

A quick tip that you can put into practice: you can use *Pascal case* - creating names by concatenating capitalized words, such as PenguinsSubset, or PenguinsTidy. If names get too long, remove vowels! For example, PngnSubset, or PngnTidy instead. Or, you can use snake_case!

## QUESTION 1.1

{points: 1}

In the next few questions, you will practice using the dplyr verb `select()` to pick and modify variables by their names. Modify the penguins data so that it contains the columns `species`, `island`, `sex`, in that order.

Assign your answer to a variable named `answer1.1`.

In [ ]:
# answer1.1 <- select(penguins, FILL_THIS_IN)
# your code here
fail() # No Answer - remove if you provide an answer
head(answer1.1)

In [ ]:
library(digest)
stopifnot("answer1.1 should be a data frame"= setequal(digest(paste(toString('data.frame' %in% class(answer1.1)), "86bf6")), "9cdf74424c85c62fd4a5a2864846873c"))
stopifnot("dimensions of answer1.1 are not correct"= setequal(digest(paste(toString(dim(answer1.1)), "86bf6")), "c6ae35d6df7bd829ea25bf7b488ebad9"))
stopifnot("column names of answer1.1 are not correct"= setequal(digest(paste(toString(sort(colnames(answer1.1))), "86bf6")), "9c1b0cb2aed059b23d54800a9a2ce776"))
stopifnot("types of columns in answer1.1 are not correct"= setequal(digest(paste(toString(sort(unlist(sapply(answer1.1, class)))), "86bf6")), "7b20e0576766f137a6480621a4c36445"))
stopifnot("values in one or more numerical columns in answer1.1 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.1, is.numeric))) sort(round(sapply(answer1.1[, sapply(answer1.1, is.numeric)], sum, na.rm = TRUE), 2)) else 0), "86bf6")), "4c752b60f15eca0f20a9aa723ff15f8e"))
stopifnot("values in one or more character columns in answer1.1 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.1, is.character))) sum(sapply(answer1.1[sapply(answer1.1, is.character)], function(x) length(unique(x)))) else 0), "86bf6")), "4c752b60f15eca0f20a9aa723ff15f8e"))
stopifnot("values in one or more factor columns in answer1.1 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.1, is.factor))) sum(sapply(answer1.1[, sapply(answer1.1, is.factor)], function(col) length(unique(col)))) else 0), "86bf6")), "725c3bafc1f7483809d88d11330ea100"))

print('Success!')

## QUESTION 1.2

{points: 1}

Out of the following options, what would be the best name for the object that you just created above (currently stored in `answer1.1`)? Put your answer in a variable named `answer1.2`.

A) _penguin_subset   

B) penguins  

C) 2penguin   

D) PngnSub   

In [ ]:
# answer1.2 <- "FILL_THIS_IN"
# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
library(digest)
stopifnot("type of answer1.2 is not character"= setequal(digest(paste(toString(class(answer1.2)), "70f39")), "bd6ff90de7ae34f01036d66c01f7a38a"))
stopifnot("length of answer1.2 is not correct"= setequal(digest(paste(toString(length(answer1.2)), "70f39")), "452c27ebce502d0d700604c0025662c4"))
stopifnot("value of answer1.2 is not correct"= setequal(digest(paste(toString(tolower(answer1.2)), "70f39")), "778ee8a1450b40e69270ec50c9c57111"))
stopifnot("letters in string value of answer1.2 are correct but case is not correct"= setequal(digest(paste(toString(answer1.2), "70f39")), "34c817b9783ee4a2bb1fe182a7223e79"))

print('Success!')

## QUESTION 1.3

{points: 1}

Select all variables, from `bill_length_mm` to `body_mass_g` (in that order). Of course, you could do it this way...

In [ ]:
# This will work:
select(penguins, bill_length_mm, bill_depth_mm, flipper_length_mm, body_mass_g) %>% 
   print(n = 5)

But there is a better way to do it! Which do you think would work?

A) `select(penguins, body_mass_g:bill_length_mm)`   

B) `select(penguins, c(body_mass_g::bill_length_mm))`   

C) `select(penguins, bill_length_mm:body_mass_g)`   

D) `select(penguins, bill_length_mm::body_mass_g)`

Assign your answer to a variable called `answer1.3`

In [ ]:
# answer1.3 <- "FILL_THIS_IN"
# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
library(digest)
stopifnot("type of answer1.3 is not character"= setequal(digest(paste(toString(class(answer1.3)), "d9a13")), "e722e5fea3ffca8a19fa8d58a8ec897b"))
stopifnot("length of answer1.3 is not correct"= setequal(digest(paste(toString(length(answer1.3)), "d9a13")), "ece5e2b8e7318d1c702e3c0fe5312bae"))
stopifnot("value of answer1.3 is not correct"= setequal(digest(paste(toString(tolower(answer1.3)), "d9a13")), "3bc96418c2bc7e587d6752f24cf8e8ec"))
stopifnot("letters in string value of answer1.3 are correct but case is not correct"= setequal(digest(paste(toString(answer1.3), "d9a13")), "fa0567292f25643d9fa869f3cba99a60"))

print('Success!')

## QUESTION 1.4

{points: 1}

Now, select all variables, except `island`. How would you write this code?

A) `select(penguins, "-island")`   

B) `select(penguins, -island)`   

C) `select(penguins, c("-island"))`   

Put your answer in a variable named `answer1.4`. We encourage you to try executing these!

In [ ]:
# answer1.4 <- "FILL_THIS_IN"
# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
library(digest)
stopifnot("type of answer1.4 is not character"= setequal(digest(paste(toString(class(answer1.4)), "7f60b")), "439d25fd987cb848930a324e5036c7c4"))
stopifnot("length of answer1.4 is not correct"= setequal(digest(paste(toString(length(answer1.4)), "7f60b")), "1242860c7ae6a36a43e270af26781b43"))
stopifnot("value of answer1.4 is not correct"= setequal(digest(paste(toString(tolower(answer1.4)), "7f60b")), "a5990bbc986f8177d2cab51c74f4d81e"))
stopifnot("letters in string value of answer1.4 are correct but case is not correct"= setequal(digest(paste(toString(answer1.4), "7f60b")), "05207e19e3f1a96f69826dcd620b4752"))

print('Success!')

## QUESTION 1.5

{points: 1}

Output the `penguins` tibble so that `year` comes first. Hint: use the tidyselect `everything()` function. Store the result in a variable named `answer1.5`. 

In [ ]:
# answer1.5 <- select(penguins, FILL_THIS_IN, FILL_THIS_IN)
# your code here
fail() # No Answer - remove if you provide an answer
head(answer1.5)

In [ ]:
library(digest)
stopifnot("type of dim(answer1.5) is not integer"= setequal(digest(paste(toString(class(dim(answer1.5))), "56e96")), "c115dd5b7b57399fb416476849396bd9"))
stopifnot("length of dim(answer1.5) is not correct"= setequal(digest(paste(toString(length(dim(answer1.5))), "56e96")), "7e85ce0493a51f964864ea4de12b610d"))
stopifnot("values of dim(answer1.5) are not correct"= setequal(digest(paste(toString(sort(dim(answer1.5))), "56e96")), "45b94405c4f06e4b7ddda07070e3bfc1"))

stopifnot("answer1.5 should be a data frame"= setequal(digest(paste(toString('data.frame' %in% class(answer1.5)), "56e97")), "9bea5968c1d59783fbdbb6cd29e775fc"))
stopifnot("dimensions of answer1.5 are not correct"= setequal(digest(paste(toString(dim(answer1.5)), "56e97")), "7f3b3f5f3a4b0ad703fafb24e9c7b056"))
stopifnot("column names of answer1.5 are not correct"= setequal(digest(paste(toString(sort(colnames(answer1.5))), "56e97")), "476b4dec392e4711ec6392eb0392f7d5"))
stopifnot("types of columns in answer1.5 are not correct"= setequal(digest(paste(toString(sort(unlist(sapply(answer1.5, class)))), "56e97")), "c62c536a5821283e6052a3f82d8b1962"))
stopifnot("values in one or more numerical columns in answer1.5 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.5, is.numeric))) sort(round(sapply(answer1.5[, sapply(answer1.5, is.numeric)], sum, na.rm = TRUE), 2)) else 0), "56e97")), "1120a49a7b13ae70759cca64d9948ca7"))
stopifnot("values in one or more character columns in answer1.5 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.5, is.character))) sum(sapply(answer1.5[sapply(answer1.5, is.character)], function(x) length(unique(x)))) else 0), "56e97")), "e7d6da79d9ee995dca0110647d347a47"))
stopifnot("values in one or more factor columns in answer1.5 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.5, is.factor))) sum(sapply(answer1.5[, sapply(answer1.5, is.factor)], function(col) length(unique(col)))) else 0), "56e97")), "647efa9196b158ffc8a4d26ec191a1d0"))

print('Success!')

## QUESTION 1.6

{points: 1}

Rename `flipper_length_mm` to `length_flipper_mm`. Store the result in a variable named `answer1.6`

In [ ]:
# answer1.6 <- rename(FILL_THIS_IN, FILL_THIS_IN)
# your code here
fail() # No Answer - remove if you provide an answer
head(answer1.6)

In [ ]:
library(digest)
stopifnot("answer1.6 should be a data frame"= setequal(digest(paste(toString('data.frame' %in% class(answer1.6)), "2b5f9")), "e7aecfe396fee454e4823dc65c247c4b"))
stopifnot("dimensions of answer1.6 are not correct"= setequal(digest(paste(toString(dim(answer1.6)), "2b5f9")), "7da673f2efe24d7ed5bb99995f2ea23f"))
stopifnot("column names of answer1.6 are not correct"= setequal(digest(paste(toString(sort(colnames(answer1.6))), "2b5f9")), "5dcd43867fc4ca0ed66d126765d2dfb2"))
stopifnot("types of columns in answer1.6 are not correct"= setequal(digest(paste(toString(sort(unlist(sapply(answer1.6, class)))), "2b5f9")), "7f3177f5328bb2e46da2225bbb0488d4"))
stopifnot("values in one or more numerical columns in answer1.6 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.6, is.numeric))) sort(round(sapply(answer1.6[, sapply(answer1.6, is.numeric)], sum, na.rm = TRUE), 2)) else 0), "2b5f9")), "6d322e842542771bab10744cb106e7a0"))
stopifnot("values in one or more character columns in answer1.6 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.6, is.character))) sum(sapply(answer1.6[sapply(answer1.6, is.character)], function(x) length(unique(x)))) else 0), "2b5f9")), "1377b620d7acc7fea093fc4fca479f76"))
stopifnot("values in one or more factor columns in answer1.6 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.6, is.factor))) sum(sapply(answer1.6[, sapply(answer1.6, is.factor)], function(col) length(unique(col)))) else 0), "2b5f9")), "a7aa8f613cf91e06d1d5356b62318e51"))

print('Success!')

## `filter()` 

So far, we've practiced picking variables by their name with `select()`. But how about picking observations (rows)? This is where `filter()` comes in.

## QUESTION 1.7

{points: 1}

Pick penguins with body mass greater than 3600 g. Store the resulting tibble in a variable named `answer1.7`

In [ ]:
# answer1.7 <- filter(FILL_THIS_IN, FILL_THIS_IN)
# your code here
fail() # No Answer - remove if you provide an answer
head(answer1.7)

In [ ]:
library(digest)
stopifnot("answer1.7 should be a data frame"= setequal(digest(paste(toString('data.frame' %in% class(answer1.7)), "e6d5d")), "93f9389800c6f6fbdd58623076de1dd7"))
stopifnot("dimensions of answer1.7 are not correct"= setequal(digest(paste(toString(dim(answer1.7)), "e6d5d")), "a4525a27572138f5bd22bf46e05c0e89"))
stopifnot("column names of answer1.7 are not correct"= setequal(digest(paste(toString(sort(colnames(answer1.7))), "e6d5d")), "582187f5fddbeae59a165a470e157df6"))
stopifnot("types of columns in answer1.7 are not correct"= setequal(digest(paste(toString(sort(unlist(sapply(answer1.7, class)))), "e6d5d")), "e498dbda396811da6adb153331540db2"))
stopifnot("values in one or more numerical columns in answer1.7 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.7, is.numeric))) sort(round(sapply(answer1.7[, sapply(answer1.7, is.numeric)], sum, na.rm = TRUE), 2)) else 0), "e6d5d")), "470fa5e258752f355ee032004767cf4b"))
stopifnot("values in one or more character columns in answer1.7 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.7, is.character))) sum(sapply(answer1.7[sapply(answer1.7, is.character)], function(x) length(unique(x)))) else 0), "e6d5d")), "dd954b7e2d40b4b4b6c61a90fae2b99e"))
stopifnot("values in one or more factor columns in answer1.7 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.7, is.factor))) sum(sapply(answer1.7[, sapply(answer1.7, is.factor)], function(col) length(unique(col)))) else 0), "e6d5d")), "9f8b6d5eadc85f0a7463f53339ec8dec"))

print('Success!')

## Storing the subsetted penguins data

In question 1.7 above, you've created a subset of the `penguins` dataset by filtering for those penguins that have a body mass greater than 3600 g. Let's do a quick check to see how many penguins meet that threshold by comparing the dimensions of the `penguins` dataset and your subset, `answer1.7`. There are two different ways to do this. 

In [ ]:
dim(penguins)
dim(answer1.7)

As you can see, in filtering down to penguins with a body mass greater than 3600g, we have lost about 100 rows (observations). However, `answer1.7` doesn't seem like an informative name for this new dataset that you've created from `penguins`. Let's rename it to something else.

In [ ]:
penguins3600 <- answer1.7

## QUESTION 1.8

{points: 1}

From your "new" dataset `penguins3600`, take only data from penguins located in the Biscoe island. Store the result in a variable named `answer1.8`. 

In [ ]:
# answer1.8 <- filter(FILL_THIS_IN, FILL_THIS_IN)
# your code here
fail() # No Answer - remove if you provide an answer
head(answer1.8)

In [ ]:
library(digest)
stopifnot("answer1.8 should be a data frame"= setequal(digest(paste(toString('data.frame' %in% class(answer1.8)), "9b567")), "497251b5383f92dec76bbf28cd6fe300"))
stopifnot("dimensions of answer1.8 are not correct"= setequal(digest(paste(toString(dim(answer1.8)), "9b567")), "880ed31b35f5c8cd9f5132c03d20389d"))
stopifnot("column names of answer1.8 are not correct"= setequal(digest(paste(toString(sort(colnames(answer1.8))), "9b567")), "b90ed6d51aea34ba1b99f7fe919c663d"))
stopifnot("types of columns in answer1.8 are not correct"= setequal(digest(paste(toString(sort(unlist(sapply(answer1.8, class)))), "9b567")), "3cc42525072726923d7a58d4c994cd51"))
stopifnot("values in one or more numerical columns in answer1.8 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.8, is.numeric))) sort(round(sapply(answer1.8[, sapply(answer1.8, is.numeric)], sum, na.rm = TRUE), 2)) else 0), "9b567")), "12960c4b930771e79fede0fac626ef47"))
stopifnot("values in one or more character columns in answer1.8 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.8, is.character))) sum(sapply(answer1.8[sapply(answer1.8, is.character)], function(x) length(unique(x)))) else 0), "9b567")), "e0cd1407fc738e531f552d4213d5b015"))
stopifnot("values in one or more factor columns in answer1.8 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.8, is.factor))) sum(sapply(answer1.8[, sapply(answer1.8, is.factor)], function(col) length(unique(col)))) else 0), "9b567")), "c525da4d4e0c9c6efbd3ef47741a5253"))

print('Success!')

## QUESTION 1.9

{points: 1}

Repeat the task from Question 1.8, but take data from islands Torgersen and Dream. Now that you've practiced with dplyr verbs quite a bit, you don't need as many prompts to answer! Hint: When you want to select more than one island, you use `%in%` instead of `==`.

Store your answer in a variable named `answer1.9`.

In [ ]:
# answer1.9 <- FILL_THIS_IN(FILL_THIS_IN, island FILL_THIS_IN c("FILL_THIS_IN", "FILL_THIS_IN"))
# your code here
fail() # No Answer - remove if you provide an answer
head(answer1.9)

In [ ]:
library(digest)
stopifnot("answer1.9 should be a data frame"= setequal(digest(paste(toString('data.frame' %in% class(answer1.9)), "753df")), "b0aeedeb0533157a8efb729228a32fb8"))
stopifnot("dimensions of answer1.9 are not correct"= setequal(digest(paste(toString(dim(answer1.9)), "753df")), "62729e8d6491c49a37cbd16cbbd7dc8c"))
stopifnot("column names of answer1.9 are not correct"= setequal(digest(paste(toString(sort(colnames(answer1.9))), "753df")), "3fab85aaa09152527548e8907948a60b"))
stopifnot("types of columns in answer1.9 are not correct"= setequal(digest(paste(toString(sort(unlist(sapply(answer1.9, class)))), "753df")), "bd05bd85b12989b15bd4ba7b7c040ec6"))
stopifnot("values in one or more numerical columns in answer1.9 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.9, is.numeric))) sort(round(sapply(answer1.9[, sapply(answer1.9, is.numeric)], sum, na.rm = TRUE), 2)) else 0), "753df")), "265fd037fbc38ca3689d9f784fc8796f"))
stopifnot("values in one or more character columns in answer1.9 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.9, is.character))) sum(sapply(answer1.9[sapply(answer1.9, is.character)], function(x) length(unique(x)))) else 0), "753df")), "27e944b0b52a8d7b47babf5757f307e6"))
stopifnot("values in one or more factor columns in answer1.9 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.9, is.factor))) sum(sapply(answer1.9[, sapply(answer1.9, is.factor)], function(col) length(unique(col)))) else 0), "753df")), "0f4cacb5b2ac1e01395dbb28a3492227"))

print('Success!')

## `arrange()` 

`arrange()` allows you to rearrange rows. Let's give it a try!

## QUESTION 1.10

{points: 1}

Order `penguins` by year, in ascending order. Store the resulting tibble in a variable named `answer1.10`.

In [ ]:
# answer1.10 <- arrange(FILL_THIS_IN, FILL_THIS_IN)
# your code here
fail() # No Answer - remove if you provide an answer
head(answer1.10)

In [ ]:
library(digest)
stopifnot("answer1.10 should be a data frame"= setequal(digest(paste(toString('data.frame' %in% class(answer1.10)), "9ed0a")), "39fc469020732e076f6bd43988cdfbd1"))
stopifnot("dimensions of answer1.10 are not correct"= setequal(digest(paste(toString(dim(answer1.10)), "9ed0a")), "d1f92b5605580e760ad0b54ba92233cf"))
stopifnot("column names of answer1.10 are not correct"= setequal(digest(paste(toString(sort(colnames(answer1.10))), "9ed0a")), "c289112f7d59322bf22ed5c3272c78de"))
stopifnot("types of columns in answer1.10 are not correct"= setequal(digest(paste(toString(sort(unlist(sapply(answer1.10, class)))), "9ed0a")), "681ec5b8d7c12f75761ab63803a36419"))
stopifnot("values in one or more numerical columns in answer1.10 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.10, is.numeric))) sort(round(sapply(answer1.10[, sapply(answer1.10, is.numeric)], sum, na.rm = TRUE), 2)) else 0), "9ed0a")), "4503c6000d9ec6ac0f84cb9e0b81e8f7"))
stopifnot("values in one or more character columns in answer1.10 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.10, is.character))) sum(sapply(answer1.10[sapply(answer1.10, is.character)], function(x) length(unique(x)))) else 0), "9ed0a")), "bf66de0b1082ca0b60f8e0530c3689d4"))
stopifnot("values in one or more factor columns in answer1.10 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.10, is.factor))) sum(sapply(answer1.10[, sapply(answer1.10, is.factor)], function(col) length(unique(col)))) else 0), "9ed0a")), "b152cfa7f0837c889f09de9a51c2889c"))

print('Success!')

## QUESTION 1.11

{points: 1}

Great work! Order `penguins` by year, in descending order. Hint: there is a function that allows you to order a variable in descending order called `desc()`.

Store your tibble in a variable named `answer1.11`.

In [ ]:
# answer1.11 <- arrange(FILL_THIS_IN, FILL_THIS_IN)
# your code here
fail() # No Answer - remove if you provide an answer
head(answer1.11)

In [ ]:
library(digest)
stopifnot("answer1.11 should be a data frame"= setequal(digest(paste(toString('data.frame' %in% class(answer1.11)), "4c4d8")), "55d421b6b5a1cfd1e54ed103c6f998b1"))
stopifnot("dimensions of answer1.11 are not correct"= setequal(digest(paste(toString(dim(answer1.11)), "4c4d8")), "781e91b980b1a1c65318b9823b76c6cf"))
stopifnot("column names of answer1.11 are not correct"= setequal(digest(paste(toString(sort(colnames(answer1.11))), "4c4d8")), "f79d5992de6928aba3240d02a66504d9"))
stopifnot("types of columns in answer1.11 are not correct"= setequal(digest(paste(toString(sort(unlist(sapply(answer1.11, class)))), "4c4d8")), "777891d973f8714fc0c24966566aac91"))
stopifnot("values in one or more numerical columns in answer1.11 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.11, is.numeric))) sort(round(sapply(answer1.11[, sapply(answer1.11, is.numeric)], sum, na.rm = TRUE), 2)) else 0), "4c4d8")), "f8a072d47da47344f08ea610217ae412"))
stopifnot("values in one or more character columns in answer1.11 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.11, is.character))) sum(sapply(answer1.11[sapply(answer1.11, is.character)], function(x) length(unique(x)))) else 0), "4c4d8")), "89c3e6580704b69df5c7274a73d7d570"))
stopifnot("values in one or more factor columns in answer1.11 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.11, is.factor))) sum(sapply(answer1.11[, sapply(answer1.11, is.factor)], function(col) length(unique(col)))) else 0), "4c4d8")), "ef47c12f78580f02bbb5dfd742313ccf"))

print('Success!')

## QUESTION 1.12

{points: 1}

Order `penguins` by year, then by `body_mass_g`. Use ascending order in both cases.

Store your answer in a variable named `answer1.12`

In [ ]:
# answer1.12 <- arrange(FILL_THIS_IN, FILL_THIS_IN, FILL_THIS_IN)
# your code here
fail() # No Answer - remove if you provide an answer
head(answer1.12)

In [ ]:
library(digest)
stopifnot("answer1.12 should be a data frame"= setequal(digest(paste(toString('data.frame' %in% class(answer1.12)), "61a79")), "915b010d94af589535213bbfee5ba59a"))
stopifnot("dimensions of answer1.12 are not correct"= setequal(digest(paste(toString(dim(answer1.12)), "61a79")), "446421959593cdf585db540cd456e3af"))
stopifnot("column names of answer1.12 are not correct"= setequal(digest(paste(toString(sort(colnames(answer1.12))), "61a79")), "0dc065ddd7d77b7f6da832a1269be507"))
stopifnot("types of columns in answer1.12 are not correct"= setequal(digest(paste(toString(sort(unlist(sapply(answer1.12, class)))), "61a79")), "245980453591a43d0fce5c5f49fed61f"))
stopifnot("values in one or more numerical columns in answer1.12 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.12, is.numeric))) sort(round(sapply(answer1.12[, sapply(answer1.12, is.numeric)], sum, na.rm = TRUE), 2)) else 0), "61a79")), "50bf3c1d57833e96254db26eaf03d4ae"))
stopifnot("values in one or more character columns in answer1.12 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.12, is.character))) sum(sapply(answer1.12[sapply(answer1.12, is.character)], function(x) length(unique(x)))) else 0), "61a79")), "759916d9028a9c11d5c797d4b48ab00a"))
stopifnot("values in one or more factor columns in answer1.12 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.12, is.factor))) sum(sapply(answer1.12[, sapply(answer1.12, is.factor)], function(col) length(unique(col)))) else 0), "61a79")), "4ade720203e058a904cb5c68dc807b18"))

print('Success!')

## Piping, `%>%` 

So far, we've been using dplyr verbs by inputting the dataset that we want to work on as the first argument of the function (e.g. `select(**penguins**, year))`. This is fine when you're using a single verb, i.e. you only want to filter observations, or select variables. However, more often than not you will want to do several tasks at once; such as filtering penguins with a certain body mass, and simultaneously ordering those penguins by year. Here is where piping (`%>%`) comes in.

Think of `%>%` as the word "then"!

Let's see an example. Here I want to combine `select()` with `arrange()`.

This is how I could do it by *nesting* the two function calls. I am selecting variables year, species, island, and body_mass_g, while simultaneously arranging by year.

In [ ]:
print(arrange(select(penguins, year, species, island, body_mass_g), year), n = 5)

However, that seems a little hard to read. Now using pipes:

In [ ]:
penguins %>%
  select(year, species, island, body_mass_g) %>%
  arrange(year) %>% 
  print(n = 5)

## Creating tibbles

Throughout Part A, we have been working with a tibble, `penguins`. Remember that when we ran `class()` on `penguins`, we could see that it was a dataframe that had been coerced to a tibble, which is a unifying feature of the tidyverse.

Suppose that you have a dataframe that you want to coerce to a tibble. To do this, you can use `as_tibble()`. R comes with a few built-in datasets, one of which is `mtcars`. Let's check the class of `mtcars`:

In [ ]:
class(mtcars)

As you can see, mtcars is a dataframe. Now, coerce it to a tibble with `as_tibble()`:

In [ ]:
as_tibble(mtcars) %>% 
    print(n = 5)

You can read more about tibbles in the [R4DS Tibble Chapter](https://r4ds.had.co.nz/tibbles.html#creating-tibbles).


## QUESTION 1.13

{points: 1}

At the start of this worksheet, we loaded a package called `gapminder`. This package comes with a dataset stored in the variable also named `gapminder`. Check the class of the `gapminder` dataset:

In [ ]:
class(gapminder)

As you can see, it is already a tibble.

Take all countries in Europe that have a GDP per capita greater than 10000, and select all variables except `gdpPercap`, using pipes. (Hint: use `-`).

Store your answer in a variable named `answer1.13`. Here is a code snippet that you can copy and paste into the solution cell below. 

```
answer1.13 <- FILL_THIS_IN %>%
  filter(FILL_THIS_IN > 10000, FILL_THIS_IN == "Europe") %>%
  FILL_THIS_IN(-FILL_THIS_IN)
```

In [ ]:
# your code here
fail() # No Answer - remove if you provide an answer
head(answer1.13)

In [ ]:
library(digest)
stopifnot("answer1.13 should be a data frame"= setequal(digest(paste(toString('data.frame' %in% class(answer1.13)), "25b53")), "f5540a5f57baac069b0d184335f410d1"))
stopifnot("dimensions of answer1.13 are not correct"= setequal(digest(paste(toString(dim(answer1.13)), "25b53")), "4de41592a6202803fb87d9a53a5c23de"))
stopifnot("column names of answer1.13 are not correct"= setequal(digest(paste(toString(sort(colnames(answer1.13))), "25b53")), "d5394b44e2be03dac6af45c968de83c7"))
stopifnot("types of columns in answer1.13 are not correct"= setequal(digest(paste(toString(sort(unlist(sapply(answer1.13, class)))), "25b53")), "0aedb6953476be6f2b53d21cc816dca7"))
stopifnot("values in one or more numerical columns in answer1.13 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.13, is.numeric))) sort(round(sapply(answer1.13[, sapply(answer1.13, is.numeric)], sum, na.rm = TRUE), 2)) else 0), "25b53")), "0ffdc2b6931c5d26c40dd4644b2d545a"))
stopifnot("values in one or more character columns in answer1.13 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.13, is.character))) sum(sapply(answer1.13[sapply(answer1.13, is.character)], function(x) length(unique(x)))) else 0), "25b53")), "a364e1297a7e991c2d715fe32c536543"))
stopifnot("values in one or more factor columns in answer1.13 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.13, is.factor))) sum(sapply(answer1.13[, sapply(answer1.13, is.factor)], function(col) length(unique(col)))) else 0), "25b53")), "22c23aaaa6c6e58411afc18c5eebc424"))

print('Success!')

## QUESTION 1.14

{points: 1}

Coerce the `mtcars` data frame to a tibble, and take all columns that start with the letter "d". 
*Hint: take a look at the "Select helpers" documentation by running the following code: `?tidyselect::select_helpers`.*

Store your tibble in a variable named `answer1.14`

```
answer1.14 <- FILL_THIS_IN(FILL_THIS_IN) %>%
    FILL_THIS_IN(FILL_THIS_IN("d"))
```

In [ ]:
# your code here
fail() # No Answer - remove if you provide an answer
head(answer1.14)

In [ ]:
library(digest)
stopifnot("answer1.14 should be a data frame"= setequal(digest(paste(toString('data.frame' %in% class(answer1.14)), "1b07b")), "ba86ca2008a79291b457dede6afc06e6"))
stopifnot("dimensions of answer1.14 are not correct"= setequal(digest(paste(toString(dim(answer1.14)), "1b07b")), "fd73cb0537053520e0b03f42bb0992ab"))
stopifnot("column names of answer1.14 are not correct"= setequal(digest(paste(toString(sort(colnames(answer1.14))), "1b07b")), "1bbdd6640428bf09cf588f38d76e2efb"))
stopifnot("types of columns in answer1.14 are not correct"= setequal(digest(paste(toString(sort(unlist(sapply(answer1.14, class)))), "1b07b")), "88a8fa4235a5ba6bb8680a91fb41e092"))
stopifnot("values in one or more numerical columns in answer1.14 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.14, is.numeric))) sort(round(sapply(answer1.14[, sapply(answer1.14, is.numeric)], sum, na.rm = TRUE), 2)) else 0), "1b07b")), "510f8873ae245e3cc33d2574e85e2166"))
stopifnot("values in one or more character columns in answer1.14 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.14, is.character))) sum(sapply(answer1.14[sapply(answer1.14, is.character)], function(x) length(unique(x)))) else 0), "1b07b")), "97e4cf2a540632d95a62223bb7e372a6"))
stopifnot("values in one or more factor columns in answer1.14 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.14, is.factor))) sum(sapply(answer1.14[, sapply(answer1.14, is.factor)], function(col) length(unique(col)))) else 0), "1b07b")), "97e4cf2a540632d95a62223bb7e372a6"))

print('Success!')

This exercise is from [r-exercises](https://www.r-exercises.com/2017/10/19/dplyr-basic-functions-exercises/).

## `mutate()`

The `mutate()` function allows you to create new columns, possibly using existing columns. Like `select()`, `filter()`, and `arrange()`, the `mutate()` function also takes a tibble as its first argument, and returns a tibble. 

The general syntax is: `mutate(tibble, NEW_COLUMN_NAME = CALCULATION)`.

## QUESTION 1.15

{points: 1}

Make a new column with body mass in kg, named `body_mass_kg`, *and* rearrange the tibble so that `body_mass_kg` goes after `body_mass_g` and before `sex`. Store the resulting tibble in a variable named `answer1.15`.


*Hint*: within `select()`, use R's `:` operator to select all variables from `species` to `body_mass_g`.

```
answer1.15 <- penguins %>%
    mutate(FILL_THIS_IN = FILL_THIS_IN) %>%
    select(FILL_THIS_IN, FILL_THIS_IN, FILL_THIS_IN, FILL_THIS_IN)
```

In [ ]:
# Your code here
# your code here
fail() # No Answer - remove if you provide an answer
head(answer1.15)

In [ ]:
library(digest)
stopifnot("answer1.15 should be a data frame"= setequal(digest(paste(toString('data.frame' %in% class(answer1.15)), "7d3a1")), "0fefab599be43c1dbc1e9f60fd0cd736"))
stopifnot("dimensions of answer1.15 are not correct"= setequal(digest(paste(toString(dim(answer1.15)), "7d3a1")), "e2085a6817639bbdcd97a7a0ae979a29"))
stopifnot("column names of answer1.15 are not correct"= setequal(digest(paste(toString(sort(colnames(answer1.15))), "7d3a1")), "4cdb509976bd216bacf2972c121ceb53"))
stopifnot("types of columns in answer1.15 are not correct"= setequal(digest(paste(toString(sort(unlist(sapply(answer1.15, class)))), "7d3a1")), "f2370f2159a33cfc55cd8dcbd9657dfa"))
stopifnot("values in one or more numerical columns in answer1.15 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.15, is.numeric))) sort(round(sapply(answer1.15[, sapply(answer1.15, is.numeric)], sum, na.rm = TRUE), 2)) else 0), "7d3a1")), "5a07c804c6ef1139d06c2cd77da89b66"))
stopifnot("values in one or more character columns in answer1.15 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.15, is.character))) sum(sapply(answer1.15[sapply(answer1.15, is.character)], function(x) length(unique(x)))) else 0), "7d3a1")), "1b85d55e4192aef1f86db808de2783eb"))
stopifnot("values in one or more factor columns in answer1.15 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.15, is.factor))) sum(sapply(answer1.15[, sapply(answer1.15, is.factor)], function(col) length(unique(col)))) else 0), "7d3a1")), "47a83fbdfc0af98f3517e8b7340c780a"))

print('Success!')

Notice the backwards compatibility! No need for loops! By the way, if you'd like to simultaneously create columns _and_ delete other columns, use the `transmute` function.

## `group_by()`

The `group_by()` function groups the _rows_ in your tibble according to one or more categorical variables. Just specify the columns containing the grouping variables. `mutate()` (and others) will now operate on each chunk independently. 

## QUESTION 1.16

{points: 1}

Calculate the growth in population since the first year on record _for each country_, and name the column `rel_growth`. Do this by **rearranging the following lines**, and **filling in the `FILL_THIS_IN`**. Assign your answer to a variable named `answer1.16`

*Hint*: Here's another convenience function for you: `dplyr::first()`.

```
answer1.16 <-
    mutate(rel_growth = FILL_THIS_IN) %>% 
    arrange(FILL_THIS_IN) %>% 
    gapminder %>% 
    group_by(country) %>% 
```

In [ ]:
# Your code here
# your code here
fail() # No Answer - remove if you provide an answer
head(answer1.16)

In [ ]:
library(digest)
stopifnot("answer1.16 should be a data frame"= setequal(digest(paste(toString('data.frame' %in% class(answer1.16)), "44de2")), "0815095c0c185318440a7356f1320de9"))
stopifnot("dimensions of answer1.16 are not correct"= setequal(digest(paste(toString(dim(answer1.16)), "44de2")), "56e101cbe925a30750449c7355fb990e"))
stopifnot("column names of answer1.16 are not correct"= setequal(digest(paste(toString(sort(colnames(answer1.16))), "44de2")), "333ed2482f7a183b5a7c0780e7f6ca7e"))
stopifnot("types of columns in answer1.16 are not correct"= setequal(digest(paste(toString(sort(unlist(sapply(answer1.16, class)))), "44de2")), "86dea9f523948a4b7748866e77678805"))
stopifnot("values in one or more numerical columns in answer1.16 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.16, is.numeric))) sort(round(sapply(answer1.16[, sapply(answer1.16, is.numeric)], sum, na.rm = TRUE), 2)) else 0), "44de2")), "763f121df76c443519452fff732a6aea"))
stopifnot("values in one or more character columns in answer1.16 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.16, is.character))) sum(sapply(answer1.16[sapply(answer1.16, is.character)], function(x) length(unique(x)))) else 0), "44de2")), "237e157ae59f93884151756e4bc0a778"))
stopifnot("values in one or more factor columns in answer1.16 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.16, is.factor))) sum(sapply(answer1.16[, sapply(answer1.16, is.factor)], function(col) length(unique(col)))) else 0), "44de2")), "08ae8053a0678b60924448aef7e4b5bf"))

print('Success!')

## `summarise()`

The last core dplyr verb is `summarise()`. It collapses a data frame to a single row:

In [ ]:
summarise(penguins, body_mass_mean = mean(body_mass_g, na.rm = TRUE))

*From R4DS Data Transformation:* 

> `summarise()` is not terribly useful unless we pair it with `group_by()`. This changes the unit of analysis from the complete dataset to individual groups. Then, when you use the dplyr verbs on a grouped data frame they'll be automatically applied "by group".

For example, if we applied exactly the same code to a tibble grouped by island, we get the average body mass per island:

In [ ]:
penguins %>%
  group_by(island) %>%
  summarise(body_mass_mean = mean(body_mass_g, na.rm = TRUE))

## QUESTION 1.17

{points: 1}

From the `penguins` tibble, calculate the mean penguin body mass per island by year, in a column named `body_mass_mean`. Your tibble should have the columns `year`, `island`, and `body_mass_mean` only (and in that order). Store the resulting tibble in a variable named `answer1.17`.

```
answer1.17 <- penguins %>%
  group_by(FILL_THIS_IN) %>%
  FILL_THIS_IN(body_mass_mean = mean(FILL_THIS_IN, na.rm = TRUE))
```

You may get a warning about the `summarise()` grouping. You can safely ignore this. 

In [ ]:
# Your code here
# your code here
fail() # No Answer - remove if you provide an answer
head(answer1.17)

In [ ]:
library(digest)
stopifnot("answer1.17 should be a data frame"= setequal(digest(paste(toString('data.frame' %in% class(answer1.17)), "63c17")), "f0c75f97026c751b62157a21188a92a8"))
stopifnot("dimensions of answer1.17 are not correct"= setequal(digest(paste(toString(dim(answer1.17)), "63c17")), "4cbc48c45f187c834f705a4a0b0759b4"))
stopifnot("column names of answer1.17 are not correct"= setequal(digest(paste(toString(sort(colnames(answer1.17))), "63c17")), "fe467fe996b44b24b7b4c07b152fdc2d"))
stopifnot("types of columns in answer1.17 are not correct"= setequal(digest(paste(toString(sort(unlist(sapply(answer1.17, class)))), "63c17")), "e450051f53b30be677886897cbea3b34"))
stopifnot("values in one or more numerical columns in answer1.17 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.17, is.numeric))) sort(round(sapply(answer1.17[, sapply(answer1.17, is.numeric)], sum, na.rm = TRUE), 2)) else 0), "63c17")), "2f40928b40739aa3a2bfb69631bff202"))
stopifnot("values in one or more character columns in answer1.17 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.17, is.character))) sum(sapply(answer1.17[sapply(answer1.17, is.character)], function(x) length(unique(x)))) else 0), "63c17")), "c80809bbe6850ddcce432d143da41074"))
stopifnot("values in one or more factor columns in answer1.17 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer1.17, is.factor))) sum(sapply(answer1.17[, sapply(answer1.17, is.factor)], function(col) length(unique(col)))) else 0), "63c17")), "875f889a2f5ceb7587787f82806d1f45"))

print('Success!')

# Part 2: Scoped variants with `across()`

Sometimes we want to perform the same operation on many columns. We can achieve this by embedding the `across()` function within the `mutate()` or `summarise()` functions.

## QUESTION 2.0

{points: 1}

In a single expression, make a tibble with the following columns *for each island* in the penguins data set:

+ What is the *mean* of each numeric variable in the `penguins` dataset in each island? Keep the column names the same.
+ How many penguins are there in each island? Add this to a column named `n`.

Assign your answer to a variable named `answer2.0`

```
answer2.0 <- penguins %>% 
 group_by(FILL_THIS_IN) %>% 
 summarise(across(where(FILL_THIS_IN), FILL_THIS_IN, na.rm = TRUE), 
           n = n())
```

You can ignore the warning message about `summarise()` again if one appears.

In [ ]:
# Your code here
# your code here
fail() # No Answer - remove if you provide an answer
head(answer2.0)        

In [ ]:
library(digest)
stopifnot("answer2.0 should be a data frame"= setequal(digest(paste(toString('data.frame' %in% class(answer2.0)), "bd864")), "c158cc80ab856c767cf2efcf045fcc72"))
stopifnot("dimensions of answer2.0 are not correct"= setequal(digest(paste(toString(dim(answer2.0)), "bd864")), "5303d71027cfe9afd9713376b4c9a80a"))
stopifnot("column names of answer2.0 are not correct"= setequal(digest(paste(toString(sort(colnames(answer2.0))), "bd864")), "2eb3dc8900d9763d29e0439c88a813e3"))
stopifnot("types of columns in answer2.0 are not correct"= setequal(digest(paste(toString(sort(unlist(sapply(answer2.0, class)))), "bd864")), "8a17d33f75aa06f2490f3e1a9054277d"))
stopifnot("values in one or more numerical columns in answer2.0 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer2.0, is.numeric))) sort(round(sapply(answer2.0[, sapply(answer2.0, is.numeric)], sum, na.rm = TRUE), 2)) else 0), "bd864")), "10013919fdfe8f7be71bcf7e3d37f292"))
stopifnot("values in one or more character columns in answer2.0 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer2.0, is.character))) sum(sapply(answer2.0[sapply(answer2.0, is.character)], function(x) length(unique(x)))) else 0), "bd864")), "937e9306d5d219025ad2f370c9369a60"))
stopifnot("values in one or more factor columns in answer2.0 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer2.0, is.factor))) sum(sapply(answer2.0[, sapply(answer2.0, is.factor)], function(col) length(unique(col)))) else 0), "bd864")), "d3b6026865fcdab30ea909c7a547d4e3"))

print('Success!')

## QUESTION 2.1

{points: 1}

Using the `penguins` dataset, what is the mean bill length and depth of penguins on each island, by year? The resulting tibble should have columns named `island`, `year`, `bill_length_mm`, and `bill_depth_mm`, in that order. Store the result in a variable named `answer2.1`. Be sure to remove NA's when you are calculating the mean. 

*Hint*: Use `starts_with()` instead of `where()` in the `across()` function.

```
answer2.1 <- penguins %>%
    group_by(FILL_THIS_IN) %>%
    summarise(across(FILL_THIS_IN))
```

You can again ignore the warning about `summarise()` if one appears.

In [ ]:
# Your code here
# your code here
fail() # No Answer - remove if you provide an answer
head(answer2.1)

In [ ]:
library(digest)
stopifnot("answer2.1 should be a data frame"= setequal(digest(paste(toString('data.frame' %in% class(answer2.1)), "20bd9")), "f651a4beaa52e5c84d3ce0114965ddc5"))
stopifnot("dimensions of answer2.1 are not correct"= setequal(digest(paste(toString(dim(answer2.1)), "20bd9")), "e6d9ecd062ff77e45ad4bd5d69d8df8c"))
stopifnot("column names of answer2.1 are not correct"= setequal(digest(paste(toString(sort(colnames(answer2.1))), "20bd9")), "34df2a9e952aa685f5d3682d89f24e21"))
stopifnot("types of columns in answer2.1 are not correct"= setequal(digest(paste(toString(sort(unlist(sapply(answer2.1, class)))), "20bd9")), "4d3e2bfa99dfb833462fa0cc005c1d65"))
stopifnot("values in one or more numerical columns in answer2.1 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer2.1, is.numeric))) sort(round(sapply(answer2.1[, sapply(answer2.1, is.numeric)], sum, na.rm = TRUE), 2)) else 0), "20bd9")), "a560fc515811172960d9a9b861c1eb63"))
stopifnot("values in one or more character columns in answer2.1 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer2.1, is.character))) sum(sapply(answer2.1[sapply(answer2.1, is.character)], function(x) length(unique(x)))) else 0), "20bd9")), "80530cc77803561665a0c40132f6751d"))
stopifnot("values in one or more factor columns in answer2.1 are not correct"= setequal(digest(paste(toString(if (any(sapply(answer2.1, is.factor))) sum(sapply(answer2.1[, sapply(answer2.1, is.factor)], function(col) length(unique(col)))) else 0), "20bd9")), "59f605d0c4ca38a14348f011271b74d2"))

print('Success!')

That's all! 🎉
Please Restart and Run all, check for errors, Save, and submit this worksheet on Canvas. 

## Attribution

Thanks to Icíar Fernández Boyano and Victor Yuan for their help in putting this worksheet together. 

The following resources were used as inspiration in the creation of this worksheet:

+ [Swirl R Programming Tutorial](https://swirlstats.com/scn/rprog.html)
+ [Palmer Penguins R Package](https://github.com/hadley/palmerpenguins)
+ [RD4S Data Transformation](https://r4ds.had.co.nz/transform.html)